In [1]:
from arcgis.features import SpatialDataFrame
from arcgis.geoprocessing.management import split_raster

# Split Raster

In [ ]:
split_raster(r'D:\OVERFLOW\COCONUT\5b1b6fb2-5024-4681-a175-9b667174f48c.tif', r'D:\OVERFLOW\COCONUT', 'coconut', 'SIZE_OF_TILE', 
             'jpeg', tile_size='224, 224')

In [2]:
df = SpatialDataFrame.from_featureclass(r'D:\OVERFLOW\COCONUT\Kolovai-Trees-20180108\Kolovai-Trees-20180108.shp')

In [3]:
df.tail()

,FID,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags,SHAPE
13656,13656,5322704745,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3481953, 'y': -21.1024316, 'spatial..."
13657,13657,5322704746,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3479539, 'y': -21.1023979, 'spatial..."
13658,13658,5322704747,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3478466, 'y': -21.1023253, 'spatial..."
13659,13659,5322704748,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3479297, 'y': -21.102319, 'spatialR..."
13660,13660,5322704749,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.347974, 'y': -21.1023491, 'spatialR..."


In [4]:
df.shape

(13661, 12)

In [5]:
df = df[df.other_tags.str.lower().str.contains('cocos')]

In [6]:
df.shape

(10314, 12)

In [7]:
df.head()

,FID,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags,SHAPE
94,94,5269773452,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3386164, 'y': -21.0875921, 'spatial..."
95,95,5269773559,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3385413, 'y': -21.0895804, 'spatial..."
96,96,5269773574,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3377494, 'y': -21.0882678, 'spatial..."
97,97,5269773575,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3377803, 'y': -21.0881426, 'spatial..."
98,98,5269773576,,,,,,,,,"""natural""=>""tree"",""species""=>""Cocos nucifera""","{'x': -175.3378031, 'y': -21.0882477, 'spatial..."


In [38]:
df['X'] = df.geometry.apply(lambda pt: pt['x'] )
df['Y'] = df.geometry.apply(lambda pt: pt['y'] )

In [8]:
from arcpy.sa import Raster

In [9]:
r2 = Raster(r'D:\OVERFLOW\COCONUT\coconut0.JPG')

In [10]:
r2.extent.JSON

'{"xmin":-19519769.414052021,"ymin":-2404394.878421545,"xmax":-19519750.172322504,"ymax":-2404375.636692028,"spatialReference":{"wkt":"PROJCS[\\"unnamed_ellipse_Mercator_2SP\\",GEOGCS[\\"GCS_unnamed_ellipse\\",DATUM[\\"D_unknown\\",SPHEROID[\\"Unknown\\",6378137.0,0.0]],PRIMEM[\\"Greenwich\\",0.0],UNIT[\\"Degree\\",0.0174532925199433]],PROJECTION[\\"Mercator_2SP\\"],PARAMETER[\\"false_easting\\",0.0],PARAMETER[\\"false_northing\\",0.0],PARAMETER[\\"central_meridian\\",0.0],PARAMETER[\\"standard_parallel_1\\",0.0],UNIT[\\"Meter\\",1.0]]"}}'

In [11]:
import arcpy

In [12]:
sr = arcpy.SpatialReference(4326)

In [18]:
ext = r2.extent.projectAs(sr)

In [28]:
ext.JSON

'{"xmin":-175.34907207131687,"ymin":-21.104930856349338,"xmax":-175.34889921991967,"ymax":-21.104769599295494,"spatialReference":{"wkid":4326,"latestWkid":4326}}'

In [50]:
for index, row in df.loc[(df.X > ext.XMin) & (df.X < ext.XMax) & (df.Y > ext.YMin) & (df.Y < ext.YMax)].iterrows():
    print(row['X'], row['Y'])

-175.3489745 -21.1049233


In [42]:
from pathlib import Path
folder = Path(r'D:\OVERFLOW\COCONUT')

In [64]:
tilemapping = {}
for tile in folder.glob('coconut*.JPG'):
    print(tile.name)
    path = str(tile)
    print(path)
    r = Raster(path)
    ext = r.extent.projectAs(sr)
    centers = []
    for index, row in df.loc[(df.X > ext.XMin) & (df.X < ext.XMax) & \
                             (df.Y > ext.YMin) & (df.Y < ext.YMax)].iterrows():
        x, y = row['X'], row['Y']
        x = (x - ext.XMin) / (ext.XMax - ext.XMin)
        y = (y - ext.YMin) / (ext.YMax - ext.YMin)
        print(x, y)
        centers.append((x,y))
    if len(centers) > 0:
        tilemapping[tile.name] = centers
        

coconut291.JPG
D:\OVERFLOW\COCONUT\coconut291.JPG
coconut1.JPG
D:\OVERFLOW\COCONUT\coconut1.JPG
0.061439594160909286 0.9813917938180836
coconut0.JPG
D:\OVERFLOW\COCONUT\coconut0.JPG
0.5644809267063108 0.046859031330082464
coconut2.JPG
D:\OVERFLOW\COCONUT\coconut2.JPG
0.6246320502991952 0.7953534204021965
0.3411515531008835 0.4617246040644751
0.515868104356159 0.3842086151356809
coconut3.JPG
D:\OVERFLOW\COCONUT\coconut3.JPG
0.3929220507446606 0.046859031330082464
coconut4.JPG
D:\OVERFLOW\COCONUT\coconut4.JPG
0.06748992784606639 0.34948145208747355
coconut5.JPG
D:\OVERFLOW\COCONUT\coconut5.JPG
0.774736758540929 0.9813917938180836
0.6347321049655313 0.7953534204021965
0.30496907758219205 0.05802133373371381
coconut6.JPG
D:\OVERFLOW\COCONUT\coconut6.JPG
0.20921401075510293 0.7953534204021965
0.3989723842653891 0.8189182810247411
0.9346926707888817 0.22917663727809257
coconut7.JPG
D:\OVERFLOW\COCONUT\coconut7.JPG
0.41950217421557273 0.7023342336832371
0.9315026639680652 0.7761294551441542
0

D:\OVERFLOW\COCONUT\coconut137.JPG
coconut138.JPG
D:\OVERFLOW\COCONUT\coconut138.JPG
coconut139.JPG
D:\OVERFLOW\COCONUT\coconut139.JPG
coconut140.JPG
D:\OVERFLOW\COCONUT\coconut140.JPG
coconut141.JPG
D:\OVERFLOW\COCONUT\coconut141.JPG
0.6412218443680175 0.5829152349870634
coconut142.JPG
D:\OVERFLOW\COCONUT\coconut142.JPG
coconut143.JPG
D:\OVERFLOW\COCONUT\coconut143.JPG
0.7592261305456441 0.7416678078629926
0.5457479602994598 0.2840139063876921
coconut145.JPG
D:\OVERFLOW\COCONUT\coconut145.JPG
coconut144.JPG
D:\OVERFLOW\COCONUT\coconut144.JPG
coconut146.JPG
D:\OVERFLOW\COCONUT\coconut146.JPG
coconut147.JPG
D:\OVERFLOW\COCONUT\coconut147.JPG
coconut149.JPG
D:\OVERFLOW\COCONUT\coconut149.JPG
coconut148.JPG
D:\OVERFLOW\COCONUT\coconut148.JPG
coconut150.JPG
D:\OVERFLOW\COCONUT\coconut150.JPG
coconut151.JPG
D:\OVERFLOW\COCONUT\coconut151.JPG
coconut152.JPG
D:\OVERFLOW\COCONUT\coconut152.JPG
coconut153.JPG
D:\OVERFLOW\COCONUT\coconut153.JPG
coconut154.JPG
D:\OVERFLOW\COCONUT\coconut154.JPG
c

0.2544913346249595 0.6901050039588931
coconut264.JPG
D:\OVERFLOW\COCONUT\coconut264.JPG
coconut265.JPG
D:\OVERFLOW\COCONUT\coconut265.JPG
0.9649120068070143 0.22625083781746627
coconut266.JPG
D:\OVERFLOW\COCONUT\coconut266.JPG
coconut267.JPG
D:\OVERFLOW\COCONUT\coconut267.JPG
0.3285110514734432 0.9145905763440237
0.992665359088302 0.6101087640888789
0.061229439968592404 0.30004581879751574
coconut268.JPG
D:\OVERFLOW\COCONUT\coconut268.JPG
coconut269.JPG
D:\OVERFLOW\COCONUT\coconut269.JPG
coconut270.JPG
D:\OVERFLOW\COCONUT\coconut270.JPG
0.18761435192644418 0.25477662878621044
coconut271.JPG
D:\OVERFLOW\COCONUT\coconut271.JPG
0.9486646240031095 0.053235714355636905
0.521708283440537 0.6349137997148316
coconut272.JPG
D:\OVERFLOW\COCONUT\coconut272.JPG
coconut273.JPG
D:\OVERFLOW\COCONUT\coconut273.JPG
coconut274.JPG
D:\OVERFLOW\COCONUT\coconut274.JPG
coconut275.JPG
D:\OVERFLOW\COCONUT\coconut275.JPG
coconut276.JPG
D:\OVERFLOW\COCONUT\coconut276.JPG
coconut277.JPG
D:\OVERFLOW\COCONUT\cocon

0.18139599528548672 0.9517009734241707
0.9606780967072587 0.7185341451347744
coconut413.JPG
D:\OVERFLOW\COCONUT\coconut413.JPG
coconut414.JPG
D:\OVERFLOW\COCONUT\coconut414.JPG
0.1842724877810386 0.9225551198797345
coconut415.JPG
D:\OVERFLOW\COCONUT\coconut415.JPG
coconut416.JPG
D:\OVERFLOW\COCONUT\coconut416.JPG
coconut417.JPG
D:\OVERFLOW\COCONUT\coconut417.JPG
coconut418.JPG
D:\OVERFLOW\COCONUT\coconut418.JPG
coconut419.JPG
D:\OVERFLOW\COCONUT\coconut419.JPG
coconut420.JPG
D:\OVERFLOW\COCONUT\coconut420.JPG
coconut421.JPG
D:\OVERFLOW\COCONUT\coconut421.JPG
0.017888065286695717 0.14367869351309434
0.7803927492505499 0.040738019336850756
coconut422.JPG
D:\OVERFLOW\COCONUT\coconut422.JPG
0.17205866079952614 0.5765256247378039
0.8315847154017761 0.473584950539529
coconut423.JPG
D:\OVERFLOW\COCONUT\coconut423.JPG
0.821452328950379 0.8338773101894285
0.6664058529612497 0.20569114783905246
0.08960982107060718 0.10213034911227897
coconut424.JPG
D:\OVERFLOW\COCONUT\coconut424.JPG
0.6869356427

0.5621183030423407 0.041258620684439855
coconut567.JPG
D:\OVERFLOW\COCONUT\coconut567.JPG
0.8580291472851975 0.9181128199993783
0.4466931607058028 0.9026097400914792
0.15164203095382647 0.5534803806722242
coconut568.JPG
D:\OVERFLOW\COCONUT\coconut568.JPG
coconut569.JPG
D:\OVERFLOW\COCONUT\coconut569.JPG
0.3552689978742499 0.4598417780655264
0.603459065676693 0.3823263785480625
coconut570.JPG
D:\OVERFLOW\COCONUT\coconut570.JPG
0.3561287124710012 0.6892873606231197
coconut571.JPG
D:\OVERFLOW\COCONUT\coconut571.JPG
0.889237523678486 0.9522195957879437
0.32979744064040983 0.9522195957879437
0.2233476212312893 0.23597730425010183
coconut572.JPG
D:\OVERFLOW\COCONUT\coconut572.JPG
0.3590052046962537 0.40961179916569607
coconut573.JPG
D:\OVERFLOW\COCONUT\coconut573.JPG
0.45590116938110503 0.8517596380103142
0.2771348968147985 0.5385974239676911
0.8423602961054495 0.482786336325692
0.013903006664927423 0.8517596380103142
0.2771348968147985 0.1243551289661037
coconut574.JPG
D:\OVERFLOW\COCONUT\c

...

coconut8320.JPG
D:\OVERFLOW\COCONUT\coconut8320.JPG
0.9758169132180702 0.46370866278951434
0.3822434642569165 0.11399601633473953
0.8126709946323729 0.12205676882976503
0.6732448724849185 0.6497260279119093
coconut8321.JPG
D:\OVERFLOW\COCONUT\coconut8321.JPG
coconut8322.JPG
D:\OVERFLOW\COCONUT\coconut8322.JPG
0.2057751524288886 0.34341743334325836
coconut8323.JPG
D:\OVERFLOW\COCONUT\coconut8323.JPG
0.8736818326952261 0.9287520756065161
0.1325828186782697 0.9175910337004941
0.6717742949209923 0.38000084847232074
coconut8324.JPG
D:\OVERFLOW\COCONUT\coconut8324.JPG
0.27923250318552334 0.4810702835395581
0.007322638612080184 0.09663439560672463
0.7212306661388066 0.18778290450590399
0.7270159823923328 0.19398348334992582
0.2653477441441749 0.48293045719056177
0.5505638361664996 0.9250317283045087
coconut8325.JPG
D:\OVERFLOW\COCONUT\coconut8325.JPG
0.5224969693038359 0.2113451040999696
0.043472782373864836 0.5467964192156971
0.047522503849324396 0.5467964192156971
0.25290123130901815 0.8803

0.0854405741451312 0.02315310529990043
coconut8483.JPG
D:\OVERFLOW\COCONUT\coconut8483.JPG
0.9436841594944354 0.7988438339127824
0.4820159212462775 0.31023928222516955
coconut8484.JPG
D:\OVERFLOW\COCONUT\coconut8484.JPG
0.6963538061828728 0.8955726537960639
0.44006429542817876 0.4571926816517938
coconut8485.JPG
D:\OVERFLOW\COCONUT\coconut8485.JPG
0.02611683361614021 0.49253590431543415
0.7706870372298981 0.5154779961909172
0.3905917583505002 0.07337768484324317
coconut8486.JPG
D:\OVERFLOW\COCONUT\coconut8486.JPG
0.6899737927056686 0.67483252638367
coconut8487.JPG
D:\OVERFLOW\COCONUT\coconut8487.JPG
0.8811704124637306 0.22839181926564806
0.07006907164756494 0.2166107450444346
0.4079315417085205 0.694054279055749
coconut8488.JPG
D:\OVERFLOW\COCONUT\coconut8488.JPG
0.6378897803589776 0.31023928222516955
0.242174147663399 0.7250571059270127
0.6413609700776368 0.35240312679476044
coconut8489.JPG
D:\OVERFLOW\COCONUT\coconut8489.JPG
0.43279223586881455 0.32140029990146796
coconut8490.JPG
D:\O

D:\OVERFLOW\COCONUT\coconut8575.JPG
coconut8576.JPG
D:\OVERFLOW\COCONUT\coconut8576.JPG
coconut8577.JPG
D:\OVERFLOW\COCONUT\coconut8577.JPG
coconut8578.JPG
D:\OVERFLOW\COCONUT\coconut8578.JPG
coconut8579.JPG
D:\OVERFLOW\COCONUT\coconut8579.JPG
coconut8580.JPG
D:\OVERFLOW\COCONUT\coconut8580.JPG
coconut8581.JPG
D:\OVERFLOW\COCONUT\coconut8581.JPG
coconut8582.JPG
D:\OVERFLOW\COCONUT\coconut8582.JPG
0.7621609202064491 0.807674944327923
0.7349699337206901 0.342012989948971
coconut8583.JPG
D:\OVERFLOW\COCONUT\coconut8583.JPG
0.9064964781402991 0.8033345532684647
0.8914546557495882 0.264506006860796
0.4216869748780366 0.4740848891282253
coconut8584.JPG
D:\OVERFLOW\COCONUT\coconut8584.JPG
0.67478647870096 0.694824776958237
coconut8585.JPG
D:\OVERFLOW\COCONUT\coconut8585.JPG
0.8231717582900727 0.694824776958237
0.11736317366722365 0.5280297493473737
coconut8586.JPG
D:\OVERFLOW\COCONUT\coconut8586.JPG
0.22640830221699804 0.46974449806876695
0.4942684454787443 0.15599623055056833
coconut8588.JPG

0.5460712746577059 0.2807772645084805
coconut8664.JPG
D:\OVERFLOW\COCONUT\coconut8664.JPG
0.5781716970198738 0.5132979613634937
0.9073561926464128 0.9479566506608502
0.8223120434704764 0.25721516721695115
coconut8666.JPG
D:\OVERFLOW\COCONUT\coconut8666.JPG
0.04012111842594185 0.5368600586329944
0.3311225267017927 0.24171378744423996
coconut8665.JPG
D:\OVERFLOW\COCONUT\coconut8665.JPG
0.29150119333012736 0.6763724767415965
0.7803604178237044 0.6999345740331259
0.791931050328854 0.2999989754521958
0.5906020443446198 0.04019585017870668
coconut8668.JPG
D:\OVERFLOW\COCONUT\coconut8668.JPG
0.5775608339311954 0.44385177988658325
coconut8667.JPG
D:\OVERFLOW\COCONUT\coconut8667.JPG
0.06470062968782238 0.7656604243390323
0.429175554646541 0.6918738565363365
0.8717522490278626 0.8431673232907033
0.9064641467134481 0.12576346660513835
coconut8670.JPG
D:\OVERFLOW\COCONUT\coconut8670.JPG
0.8662319500635342 0.9981811211720167
0.7962296231114068 0.6534304346489059
0.16042336544068514 0.70365490518210

D:\OVERFLOW\COCONUT\coconut8750.JPG
0.5752305418355322 0.727985291847668
0.33513991669596066 0.037244558142179494
coconut8751.JPG
D:\OVERFLOW\COCONUT\coconut8751.JPG
0.7739480727191056 0.2158202594924747
0.8439503996827433 0.39811628795377546
0.9874262430003917 0.727985291847668
0.1497124474508915 0.7974313979271146
coconut8752.JPG
D:\OVERFLOW\COCONUT\coconut8752.JPG
0.3947125083676162 0.5804123164371049
0.7713365973639262 0.7081435472629813
coconut8753.JPG
D:\OVERFLOW\COCONUT\coconut8753.JPG
0.13176180144496957 0.3943959608427699
0.4227632096729715 0.8947799573611315
coconut8754.JPG
D:\OVERFLOW\COCONUT\coconut8754.JPG
0.6330513730385271 0.029183849394324547
0.8893408836287925 0.727985291847668
0.7921475703393532 0.37889459787290386
0.6799124348236317 0.7354259460696791
0.43519355657599 0.42911901389350754
0.03947792381534448 0.07196761117088839
coconut8755.JPG
D:\OVERFLOW\COCONUT\coconut8755.JPG
0.40539109495518116 0.9059409386941482
0.8977215093318459 0.6969825659299647
coconut8756.J

In [68]:
import pickle
pickle.dump(tilemapping, open( r"D:\OVERFLOW\COCONUT\tilemapping.pkl", "wb" ) )

In [69]:
len(tilemapping)

4185

In [14]:
import arcgis

In [16]:
from arcgis.geoprocessing.analysis import spatial_join, select

In [17]:
si = df.sindex

In [24]:
index = si.intersect([ext.upperLeft.X, ext.upperLeft.Y, 
                      ext.lowerRight.X, ext.lowerRight.Y])

In [30]:
index[:2]

[13026, 13027]

In [32]:
df.loc[13026]

FID                                                       13026
osm_id                                               5316293440
name                                                           
barrier                                                        
highway                                                        
ref                                                            
address                                                        
is_in                                                          
place                                                          
man_made                                                       
other_tags        "natural"=>"tree","species"=>"Cocos nucifera"
SHAPE         {'x': -175.3472811, 'y': -21.1044215, 'spatial...
Name: 13026, dtype: object